In [191]:
import pandas as pd
import numpy as np
from math import sqrt
"""
Для формализации нечисловых параметров модели были предприняты следующие действия:
1) Округа
За начало отсчета был взят САО, которому был присвоено значение 0
Далее округа были по часовой стрелке (СВАО -> ВАО -> ЮВАО) были пронумерованы как 1 2 3,
а округа против часовой стрелки (СЗАО -> ЗАО -> ЮЗАО) были пронумерованы как -1 -2 -3
Московской области также был дан номер 0, поскольку неизвестно расстояние до округов Москвы и оно будет усреднено

2) Знаки зодиака
Так как знаки зодиака образуют кольцо для приведения их к числовым значениям можно воспользоваться следующими преобразованиями

Овен - 0
Телец - 1
Близнецы - 2
Рак - 3
Лев - 4
Дева - 5
Весы - 6
Скорпион - 5
Стрелец - 4
Козерог - 3
Водолей - 2
Рыбы - 1

Значения для некоторых пар будут неравны естественному расстоянию между знаками, но оно проявится
примерно для 12 пар из 66 возможных
3) Любимый цвет

В итоговую модель данный параметр не попал, поскольку нет даже ненаучных (по аналогии с астрологией) дисциплин,
связывающих цвет глаз человека с какими-либо другими вещами
"""
terr = {'САО': 0, 
        'МО': 0, 
        'СВАО': 1, 
        'ВАО': 2, 
        'ЮВАО': 3, 
        'СЗАО': -1, 
        'ЗАО': -2, 
        'ЮЗАО': -3}
zodiac = {'Овен': 0, 
          'Телец': 1, 
          'Близнецы': 2, 
          'Рак': 3, 
          'Лев': 4, 
          'Дева': 5, 
          'Весы': 6, 
          'Скорпион': 5, 
          'Стрелец': 4, 
          'Козерог': 3, 
          'Водолей': 2, 
          'Рыбы': 1}
teacoffee = {'Ч': 0, 'К': 1}
dataset = pd.read_csv('dataset.csv')
dataset = dataset.replace({'Округ': terr, 'Знак зодиака': zodiac, 'К/Ч': teacoffee})
dataset = dataset.drop('Любимый цвет', axis = 1)
#Нормализация данных
dataset = (dataset - dataset.min()) / (dataset.max() - dataset.min())
#dataset

In [192]:
def k_nearest(k, dataset, new_series):
    """
    k - число соседей, dataset - исследуемые данные, new_series - данные о неизвестном объекте
    
    Исключаем ответ о напитке из исследумых данных и вводим новый столбец - distances,
    в котором будет храниться расстояние от каждого известного объекта до неизвестного
    Сортируем данные по созданному столбцу и оставляем только первые k элементов
    В итоге подсчитываем количество соседей по ответу о напитке и возвращаем его численное значение
    """
    features = dataset.copy()
    features = features.drop('К/Ч', axis = 1)
    dataset['distances'] = np.linalg.norm(features - new_series, axis = 1)
    knn = dataset.sort_values(by = 'distances')[:k]
    return knn.value_counts('К/Ч').index[0]

#print(k_nearest(5, dataset, [0.5, 0, 1, 0.5, 0.75]))

In [193]:
print('Введите данные о новом человеке (каждый параметр с новой строки)')
new_object = []
for feature in dataset.columns[:-1]:
    new_object.append(input(f'{feature} : '))
new_series = pd.DataFrame(new_object)
new_series = new_series.replace(terr)
new_series = new_series.replace(zodiac)
new_series = new_series.astype(np.float64)
new_series = (new_series - new_series.min()) / (new_series.max() - new_series.min())
k = int(input('Введите параметр k (сколько ближайших соседей использовать)'))
results = {0.0: 'чай', 1.0: 'кофе'}
result = k_nearest(k, dataset, new_series)
print(f'Скорее всего, человек пьет {results[result]}')

Введите данные о новом человеке (каждый параметр с новой строки)
Округ : МО
Знак зодиака : Весы
Есть работа : 0
Время подъема : 7
Продолжительность сна : 9
Введите параметр k (сколько ближайших соседей использовать)3
Скорее всего, человек пьет чай
